<a href="https://colab.research.google.com/github/DeusExMachina1993/Deep-Computer-Vision/blob/master/Project5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Project5: Image Segmentation


in this project we're going to learn how to classify each pixel on the image, the idea is to create a map of all detected object areas on the image. Basically what we want is the image below where every pixel has a label associated with it.

![alt text](https://leonardoaraujosantos.gitbooks.io/artificial-inteligence/content/image_folder_3/ImageSegmentation.PNG)

**Fully Convolutional network for segmentation**

A Fully Convolutional neural network (FCN) is a normal CNN, where the last fully connected layer is substituted by another convolution layer with a large "receptive field". The idea is to capture the global context of the scene (Tell us what we have in the image and also give some very roughe idea of the locations of things).

![alt text](https://leonardoaraujosantos.gitbooks.io/artificial-inteligence/content/image_folder_3/Fully_Convolutional_Network_Semantic.PNG)

It's important to remember that when we convert our last fully connected (FC) layer to a convolutional layer we gain some form of localization if we look at where we have more activations.

![alt text](https://leonardoaraujosantos.gitbooks.io/artificial-inteligence/content/image_folder_7/FCN.jpg)

#**Conversion from normal CNN to FCN**

Here is how we convert a normal CNN used for classification, ie: Alexnet to a FCN used for segmentation.
Just to remind us this is how Alexnet looks like:

![alt text](https://leonardoaraujosantos.gitbooks.io/artificial-inteligence/content/image_folder_7/AlexNet_2.png)

Below shows the parameters for each of the layers in AlexNet

![alt text](https://leonardoaraujosantos.gitbooks.io/artificial-inteligence/content/image_folder_7/AlexNet_1.jpg)

In Alexnet the inputs are fixed to be 224x224, so all the pooling effects will scale down the image from 224x224 to 55x55, 27x27, 13x13, then finally a single row vector on the FC layers.

![alt text](https://leonardoaraujosantos.gitbooks.io/artificial-inteligence/content/image_folder_7/AlexNet_0.jpg)

Now let's look at the steps needed to do the conversion.

1) We start with a normal CNN for classification with

![alt text](https://leonardoaraujosantos.gitbooks.io/artificial-inteligence/content/image_folder_7/FCN_CONV_1.png)

2) The second step is to convert all the FC layers to convolution layers 1x1 we don't even need to change the weights at this point. (This is already a fully convolutional neural network). The nice property of FCN networks is that we can now use any image size.

![alt text](https://leonardoaraujosantos.gitbooks.io/artificial-inteligence/content/image_folder_7/FCN_CONV_2.png)

Observe here that with a FCN we can use a different size H x N. The diagram bellow show a how a different size would appear

![alt text](https://leonardoaraujosantos.gitbooks.io/artificial-inteligence/content/image_folder_7/FCN_CONV_3.png)

3) The last step is to use a "deconv or transposed convolution" layer to recover the activation positions to something meaningful related to the image size. Imagine that we're just scaling up the activation size to the same image size.
This last "upsampling" layer also has some lernable parameters.

![alt text](https://leonardoaraujosantos.gitbooks.io/artificial-inteligence/content/image_folder_7/FCN_CONV_4.png)

Now with this structure we just need to find some "ground truth" and to end to end learning, starting from a pre-trainned network ie: Imagenet.
The problem with this approach is that we lose some resolution by just doing this because the activations were downscaled on a lot of steps.

![alt text](https://leonardoaraujosantos.gitbooks.io/artificial-inteligence/content/image_folder_7/FirstResultFCN_No_Skips.png)

To solve this problem we also get some activation from previous layers and sum/interpolate them together. This process is called "skip" from the creators of this algorithm.
Even today (2016) the winners on Imagenet on the Segmentation category, used an ensemble of FCN to win the competition.
Those up-sampling operations used on skip are also learn-able.

![alt text](https://leonardoaraujosantos.gitbooks.io/artificial-inteligence/content/image_folder_7/Skip_Layers_FCN.png)

![alt text](https://leonardoaraujosantos.gitbooks.io/artificial-inteligence/content/assets/SkipExample.png)

Below we show the effects of this "skip" process, notice how the resolution of the segmentation improves after some "skips"

![alt text](https://leonardoaraujosantos.gitbooks.io/artificial-inteligence/content/image_folder_7/AllSkips_FCN.png)

![alt text](https://leonardoaraujosantos.gitbooks.io/artificial-inteligence/content/image_folder_7/SkipConnections.png)

**Loss**

Another important point to note here is that the loss function we use in this image segmentation problem is actually still the usual loss function we use for classification: multi-class cross entropy and not something like the L2 loss like we would normally use when the output is an image.
This is because despite what you might think we are actually just assigning a class to each of our output pixels so this is a classification problem.

#**Transposed convolution layer (deconvolution "bad name")**

Basically the idea is to scale up, the scale down effect made on all previous layers.

![alt text](https://leonardoaraujosantos.gitbooks.io/artificial-inteligence/content/image_folder_3/Conv_Deconv.PNG)

![alt text](https://leonardoaraujosantos.gitbooks.io/artificial-inteligence/content/image_folder_3/Deconv_exp.PNG)

![alt text](https://leonardoaraujosantos.gitbooks.io/artificial-inteligence/content/image_folder_3/animUpsampling.gif)

It has this bad name because the upsamping forward propagation is the convolution backpropagation and the upsampling backpropagation is the convolution forward propagation.
Also in caffe source code it is wrongly called "deconvolution"

**Extreme segmentation**

There is another thing that we can do to avoid those "skiping" steps and also give better segmentation. Deconvnet also has better response for objects of different sizes.


![alt text](https://leonardoaraujosantos.gitbooks.io/artificial-inteligence/content/image_folder_7/Deconvnet.png)

This architechture is called "Deconvnet" which is basically another network but now with all convolution and pooling layers reversed. As you may suspect this is heavy, it takes 6 days to train on a TitanX. But the results are really good.
Another problem is that the trainning is made in 2 stages.
Also Deconvnets suffer less than FCN when there are small objects on the scene.
The deconvolution network output a probability map with the same size as the input.

![alt text](https://leonardoaraujosantos.gitbooks.io/artificial-inteligence/content/image_folder_7/DeconvnetResults.png)

**Unpooling**

Besides the deconvolution layer we also need now the unpooling layer. The max-pooling operation is non-invertible, but we can approximate, by recording the positions (Max Location switches) where we located the biggest values (during normal max-pool), then use this positions to reconstruct the data from the layer above (on this case a deconvolution)

![alt text](https://leonardoaraujosantos.gitbooks.io/artificial-inteligence/content/image_folder_7/UnPoolinDiagram.png)

![alt text](https://leonardoaraujosantos.gitbooks.io/artificial-inteligence/content/image_folder_7/Unpooling_1.png)

![alt text](https://leonardoaraujosantos.gitbooks.io/artificial-inteligence/content/image_folder_7/UnpoolResults.png)

In [1]:
!pip install kaggle

In [0]:
import os
import glob
import zipfile
import functools

import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
mpl.rcParams['axes.grid'] = False
mpl.rcParams['figure.figsize'] = (12,12)

from sklearn.model_selection import train_test_split
import matplotlib.image as mpimg
import pandas as pd
from PIL import Image

In [3]:
import tensorflow as tf
import tensorflow.contrib as tfcontrib
from tensorflow.python.keras import layers
from tensorflow.python.keras import losses
from tensorflow.python.keras import models
from tensorflow.python.keras import backend as K  

In [0]:
import os

# Upload the API token.
def get_kaggle_credentials():
  token_dir = os.path.join(os.path.expanduser("~"),".kaggle")
  token_file = os.path.join(token_dir, "kaggle.json")
  if not os.path.isdir(token_dir):
    os.mkdir(token_dir)
  try:
    with open(token_file,'r') as f:
      pass
  except IOError as no_file:
    try:
      from google.colab import files
    except ImportError:
      raise no_file
    
    uploaded = files.upload()
    
    if "kaggle.json" not in uploaded:
      raise ValueError("You need an API key! see: "
                       "https://github.com/Kaggle/kaggle-api#api-credentials")
    with open(token_file, "wb") as f:
      f.write(uploaded["kaggle.json"])
    os.chmod(token_file, 600)

get_kaggle_credentials()

In [0]:
import kaggle

In [0]:
competition_name = 'carvana-image-masking-challenge'

In [0]:
# Download data from Kaggle and unzip the files of interest. 
def load_data_from_zip(competition, file):
  with zipfile.ZipFile(os.path.join(competition, file), "r") as zip_ref:
    unzipped_file = zip_ref.namelist()[0]
    zip_ref.extractall(competition)

def get_data(competition):
    kaggle.api.competition_download_files(competition, competition)
    load_data_from_zip(competition, 'train.zip')
    load_data_from_zip(competition, 'train_masks.zip')
    load_data_from_zip(competition, 'train_masks.csv.zip')

In [5]:
get_data(competition_name)

NameError: ignored

In [6]:
img_dir = os.path.join(competition_name, "train")
label_dir = os.path.join(competition_name, "train_masks")

NameError: ignored

In [0]:
df_train = pd.read_csv(os.path.join(competition_name, 'train_masks.csv'))
ids_train = df_train['img'].map(lambda s: s.split('.')[0])

In [0]:
x_train_filenames = []
y_train_filenames = []
for img_id in ids_train:
  x_train_filenames.append(os.path.join(img_dir, "{}.jpg".format(img_id)))
  y_train_filenames.append(os.path.join(label_dir, "{}_mask.gif".format(img_id)))

In [0]:
x_train_filenames, x_val_filenames, y_train_filenames, y_val_filenames = \
                    train_test_split(x_train_filenames, y_train_filenames, test_size=0.2, random_state=42)

In [7]:
num_train_examples = len(x_train_filenames)
num_val_examples = len(x_val_filenames)

print("Number of training examples: {}".format(num_train_examples))
print("Number of validation examples: {}".format(num_val_examples))

NameError: ignored

In [0]:
x_train_filenames[:10]

In [0]:
y_train_filenames[:10]

In [0]:
display_num = 5

r_choices = np.random.choice(num_train_examples, display_num)

plt.figure(figsize=(10, 15))
for i in range(0, display_num * 2, 2):
  img_num = r_choices[i // 2]
  x_pathname = x_train_filenames[img_num]
  y_pathname = y_train_filenames[img_num]
  
  plt.subplot(display_num, 2, i + 1)
  plt.imshow(mpimg.imread(x_pathname))
  plt.title("Original Image")
  
  example_labels = Image.open(y_pathname)
  label_vals = np.unique(example_labels)
  
  plt.subplot(display_num, 2, i + 2)
  plt.imshow(example_labels)
  plt.title("Masked Image")  
  
plt.suptitle("Examples of Images and their Masks")
plt.show()

In [0]:
img_shape = (256, 256, 3)
batch_size = 3
epochs = 5

In [0]:
def _process_pathnames(fname, label_path):
  # We map this function onto each pathname pair  
  img_str = tf.read_file(fname)
  img = tf.image.decode_jpeg(img_str, channels=3)

  label_img_str = tf.read_file(label_path)
  # These are gif images so they return as (num_frames, h, w, c)
  label_img = tf.image.decode_gif(label_img_str)[0]
  # The label image should only have values of 1 or 0, indicating pixel wise
  # object (car) or not (background). We take the first channel only. 
  label_img = label_img[:, :, 0]
  label_img = tf.expand_dims(label_img, axis=-1)
  return img, label_img

In [0]:
def shift_img(output_img, label_img, width_shift_range, height_shift_range):
  """This fn will perform the horizontal or vertical shift"""
  if width_shift_range or height_shift_range:
      if width_shift_range:
        width_shift_range = tf.random_uniform([], 
                                              -width_shift_range * img_shape[1],
                                              width_shift_range * img_shape[1])
      if height_shift_range:
        height_shift_range = tf.random_uniform([],
                                               -height_shift_range * img_shape[0],
                                               height_shift_range * img_shape[0])
      # Translate both 
      output_img = tfcontrib.image.translate(output_img,
                                             [width_shift_range, height_shift_range])
      label_img = tfcontrib.image.translate(label_img,
                                             [width_shift_range, height_shift_range])
  return output_img, label_img

In [0]:
def flip_img(horizontal_flip, tr_img, label_img):
  if horizontal_flip:
    flip_prob = tf.random_uniform([], 0.0, 1.0)
    tr_img, label_img = tf.cond(tf.less(flip_prob, 0.5),
                                lambda: (tf.image.flip_left_right(tr_img), tf.image.flip_left_right(label_img)),
                                lambda: (tr_img, label_img))
  return tr_img, label_img

In [0]:
def _augment(img,
             label_img,
             resize=None,  # Resize the image to some size e.g. [256, 256]
             scale=1,  # Scale image e.g. 1 / 255.
             hue_delta=0,  # Adjust the hue of an RGB image by random factor
             horizontal_flip=False,  # Random left right flip,
             width_shift_range=0,  # Randomly translate the image horizontally
             height_shift_range=0):  # Randomly translate the image vertically 
  if resize is not None:
    # Resize both images
    label_img = tf.image.resize_images(label_img, resize)
    img = tf.image.resize_images(img, resize)
  
  if hue_delta:
    img = tf.image.random_hue(img, hue_delta)
  
  img, label_img = flip_img(horizontal_flip, img, label_img)
  img, label_img = shift_img(img, label_img, width_shift_range, height_shift_range)
  label_img = tf.to_float(label_img) * scale
  img = tf.to_float(img) * scale 
  return img, label_img

In [0]:
def get_baseline_dataset(filenames, 
                         labels,
                         preproc_fn=functools.partial(_augment),
                         threads=5, 
                         batch_size=batch_size,
                         shuffle=True):           
  num_x = len(filenames)
  # Create a dataset from the filenames and labels
  dataset = tf.data.Dataset.from_tensor_slices((filenames, labels))
  # Map our preprocessing function to every element in our dataset, taking
  # advantage of multithreading
  dataset = dataset.map(_process_pathnames, num_parallel_calls=threads)
  if preproc_fn.keywords is not None and 'resize' not in preproc_fn.keywords:
    assert batch_size == 1, "Batching images must be of the same size"

  dataset = dataset.map(preproc_fn, num_parallel_calls=threads)
  
  if shuffle:
    dataset = dataset.shuffle(num_x)
  
  
  # It's necessary to repeat our data for all epochs 
  dataset = dataset.repeat().batch(batch_size)
  return dataset

In [0]:
tr_cfg = {
    'resize': [img_shape[0], img_shape[1]],
    'scale': 1 / 255.,
    'hue_delta': 0.1,
    'horizontal_flip': True,
    'width_shift_range': 0.1,
    'height_shift_range': 0.1
}
tr_preprocessing_fn = functools.partial(_augment, **tr_cfg)

In [0]:
val_cfg = {
    'resize': [img_shape[0], img_shape[1]],
    'scale': 1 / 255.,
}
val_preprocessing_fn = functools.partial(_augment, **val_cfg)

In [17]:
train_ds = get_baseline_dataset(x_train_filenames,
                                y_train_filenames,
                                preproc_fn=tr_preprocessing_fn,
                                batch_size=batch_size)
val_ds = get_baseline_dataset(x_val_filenames,
                              y_val_filenames, 
                              preproc_fn=val_preprocessing_fn,
                              batch_size=batch_size)

NameError: ignored